# 在千帆 Python SDK 使用 OpenCompass 提供的功能

OpenCompass是由上海人工智能实验室开源的大模型评测平台。它涵盖了学科、语言、知识、理解、推理等五大评测维度，能够全面评估大模型的能力。OpenCompass作为一个评测工具，对于研究和开发大模型的人员来说，是非常有价值的资源。通过使用OpenCompass，用户可以更准确地了解他们的大模型在各项任务上的表现，从而进行针对性的优化和改进。

千帆 Python SDK 中内置的评估模块，支持用户使用 OpenCompass 提供的评估器和 `TEXT_POSTPROCESSORS` 对象，对模型的推理结果进行评估前的预处理，以及评估。

# 前置准备

首先，需要安装千帆 Python SDK

In [ ]:
pip install -U qianfan

然后再安装 OpenCompass。这部分的教程可以参考 OpenCompass 所提供的[官方文档](https://opencompass.org.cn/doc)，或者直接使用以下命令在 Python 中安装：

In [ ]:
!conda create -n opencompass python=3.10 pytorch torchvision torchaudio cpuonly -c pytorch -y
!conda activate opencompass
!git clone https://github.com/open-compass/opencompass opencompass
!cd opencompass
!pip install -e .

In [ ]:
from opencompass.utils import text_postprocessors

# 正文

为了在千帆 Python SDK 的评估模块中使用来自 OpenCompass 的评估器，用户需要使用 `qianfan.evaluation.opencompass_evaluator` 中提供的 `OpenCompassLocalEvaluator` 类，将 OpenCompass 评估器包装为千帆 Python SDK 的评估器。

OpenCompass 所有可以使用的评估器都存放在 `opencompass.openicl.icl_evaluator` 模块下。其中，只有那些 `score` 函数仅包含 `predictions` 和 `references` 两个参数的评估器，才能用于千帆 Python SDK 提供的 `OpenCompassLocalEvaluator` 类。这部分符合条件的评估器包括：

+ opencompass.openicl.icl_evaluator.icl_agent_evaluator.PassRateEvaluator
+ opencompass.openicl.icl_evaluator.icl_circular_evaluator.CircularEvaluator
+ opencompass.openicl.icl_evaluator.icl_em_evaluator.EMEvaluator
+ opencompass.openicl.icl_evaluator.icl_hf_evaluator.AccEvaluator
+ opencompass.openicl.icl_evaluator.icl_hf_evaluator.RougeEvaluator
+ opencompass.openicl.icl_evaluator.icl_hf_evaluator.BleuEvaluator
+ opencompass.openicl.icl_evaluator.icl_hf_evaluator.BleuFloresEvaluator
+ opencompass.openicl.icl_evaluator.icl_hf_evaluator.MccEvaluator
+ opencompass.openicl.icl_evaluator.icl_hf_evaluator.SquadEvaluator
+ opencompass.openicl.icl_evaluator.icl_jieba_rouge_evaluator.JiebaRougeEvaluator
+ opencompass.openicl.icl_evaluator.icl_toxic_evaluator.ToxicEvaluator

在本教程编写时，用户应可以直接使用上面列表所列的评估器。

而为了使用来自 OpenCompass 的 `TEXT_POSTPROCESSORS` 模块，用户应该首先对其进行一层封装：大多数 `TEXT_POSTPROCESSORS` 函数都只支持传入一个 `str` 参数作为入参。我们需要对 `TEXT_POSTPROCESSORS` 模块进行包装，使其能够额外接收 `**kwargs` 作为入参。

例如，手动编写一个 `wrapper` 函数

In [ ]:
from typing import Callable

def open_compass_text_post_processor_wrapper(fn: Callable) -> Callable:
    def wrapper(*args, **kwargs) -> str:
        
        return fn(*args)

    return wrapper

用户可以根据自身的需求，调整实际传入给 `TEXT_POSTPROCESSORS` 参数的入参。例如，有些 `TEXT_POSTPROCESSORS` 参数需要传入 `option` 参数。

下面我们演示如何使用其中的 `PassRateEvaluator` 评估器，`BleuEvaluator` 评估器和 `JiebaRougeEvaluator` 评估器来进行一次评估。

## 导入数据集

在评估之前，我们需要先导入数据集。本教程准备了一份专门用于评估的数据集，存放在 `data_file/eval_dataset.csv` 中

In [1]:
from opencompass.datasets import xsum

/opt/anaconda3/envs/opencompass/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
ds = xsum.XsumDataset.load("./data_file/opcp.jsonl")


In [10]:
from qianfan.dataset import Dataset

ds1 = Dataset.load(huggingface_dataset=ds)    
ds1.input_columns = ["dialogue"]
ds1.reference_column = "summary"

print(ds1.list(0))

[INFO][2024-07-03 15:15:01.677] dataset.py:382 [t:8266636800]: construct dataset from huggingface dataset
INFO:qianfan:construct dataset from huggingface dataset
[INFO][2024-07-03 15:15:01.680] dataset.py:399 [t:8266636800]: construct from huggingface Dataset
INFO:qianfan:construct from huggingface Dataset


{'dialogue': 'The ex-Reading defender denied fraudulent trading charges relating to the Sodje Sports Foundation - a charity to raise money for Nigerian sport.\nMr Sodje, 37, is jointly charged with elder brothers Efe, 44, Bright, 50 and Stephen, 42.\nAppearing at the Old Bailey earlier, all four denied the offence.\nThe charge relates to offences which allegedly took place between 2008 and 2014.\nSam, from Kent, Efe and Bright, of Greater Manchester, and Stephen, from Bexley, are due to stand trial in July.\nThey were all released on bail.', 'summary': 'Former Premier League footballer Sam Sodje has appeared in court alongside three brothers accused of charity fraud.'}


# 创建评估器及评估器对象

然后，我们创建评估器对象。

In [11]:
from opencompass.openicl.icl_evaluator import PassRateEvaluator, RougeEvaluator, BleuEvaluator
from qianfan.evaluation import EvaluationManager
from qianfan.evaluation.opencompass_evaluator import OpenCompassLocalEvaluator

evaluator_list = [
    OpenCompassLocalEvaluator(open_compass_evaluator=RougeEvaluator()),
    
]

em = EvaluationManager(local_evaluators=evaluator_list, pre_processors=[open_compass_text_post_processor_wrapper(xsum.Xsum_postprocess)])

# 开始评估以及保存结果

我们只需要调用 `EvaluationManager` 的 `eval_only` 函数，即可完成一次评估，并且将评估结果保存到本地的文件中

In [12]:
from qianfan.common import Prompt
template='Document：{dialogue}\n''Based on the previous text, provide a brief single summary:'
prompt = Prompt(template)


In [13]:
from qianfan.model import Service
eb_turbo_service = Service(endpoint="ernie-speed-128k")
result_ds = em.eval([eb_turbo_service], dataset = ds1, prompt_template = prompt).result_dataset

result_ds.save(data_file="eval_result.json")

[WARNING][2024-07-03 15:15:07.613] model.py:472 [t:8266636800]: service type should be specified before exec
[INFO][2024-07-03 15:15:07.615] evaluation_manager.py:497 [t:8266636800]: start to inference in batch during evaluation
INFO:qianfan:start to inference in batch during evaluation
[INFO][2024-07-03 15:15:07.616] dataset_utils.py:455 [t:11254542336]: prompt template detected, start to check template variables
INFO:qianfan:prompt template detected, start to check template variables
[INFO][2024-07-03 15:15:09.633] base.py:102 [t:11271368704]: All tasks finished, exeutor will be shutdown
INFO:qianfan:All tasks finished, exeutor will be shutdown
[INFO][2024-07-03 15:15:09.635] evaluation_manager.py:521 [t:8266636800]: start to evaluate llm 0
INFO:qianfan:start to evaluate llm 0
[INFO][2024-07-03 15:15:10.763] evaluation_manager.py:549 [t:8266636800]: start to merge evaluation result dataset
INFO:qianfan:start to merge evaluation result dataset
[INFO][2024-07-03 15:15:10.765] dataset.p

inputs: {'predictions': ['Four people, including a former Reading defender, have denied fraudulent trading charges relating to a charity set up to raise money for Nigerian sport. The case is due to stand trial in July.'], 'references': ['Former Premier League footballer Sam Sodje has appeared in court alongside three brothers accused of charity fraud.']}
{'rouge1': Score(precision=0.09090909090909091, recall=0.17647058823529413, fmeasure=0.12000000000000001), 'rouge2': Score(precision=0.0, recall=0.0, fmeasure=0.0), 'rougeL': Score(precision=0.06060606060606061, recall=0.11764705882352941, fmeasure=0.08), 'rougeLsum': Score(precision=0.06060606060606061, recall=0.11764705882352941, fmeasure=0.08)}
{'rouge1': Score(precision=0.09090909090909091, recall=0.17647058823529413, fmeasure=0.12000000000000001), 'rouge2': Score(precision=0.0, recall=0.0, fmeasure=0.0), 'rougeL': Score(precision=0.06060606060606061, recall=0.11764705882352941, fmeasure=0.08), 'rougeLsum': Score(precision=0.060606